In [3]:
pip install kafka


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install kafka-python


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from kafka import KafkaProducer

In [47]:
# Create a Kafka Producer
producer = KafkaProducer(
    bootstrap_servers = ['localhost:9092'],
    key_serializer=str.encode,
    value_serializer=str.encode,
    acks=0
)

In [24]:
# #send meesage 
# for i in range(10):
#     producer.send("my-first-topic", key="statement", value=f"hello world{i}")

In [11]:
# producer.flush()

In [56]:
#sync req
future = producer.send("my_topic", key="my_key", value="my_value")
print(future)

In [61]:
try:
    record_metadata = future.get(timeout=10)
    print(record_metadata)
except KafkaError as e:
    print(f"Failed to write message: {e}")

Failed to write message: KafkaTimeoutError: Batch for TopicPartition(topic='my_topic', partition=0) containing 2 record(s) expired: 30 seconds have passed since batch creation plus linger time


In [62]:
#async req
from kafka.errors import KafkaError

def on_send_success(record_metadata):
    print(f"Topic: {record_metadata.topic}")
    print(f"Partiton: {record_metadata.partition}")
    print(f"Offset: {record_metadata.offset}")

def on_send_error(excp):
    print(f"Failed to send message: {excp}")


producer.send("my_topic", key="my_key", value="my_value").add_callback(on_send_success).add_errback(on_send_error)


Failed to send message: KafkaTimeoutError: Batch for TopicPartition(topic='my_topic', partition=0) containing 1 record(s) expired: 30 seconds have passed since batch creation plus linger time


In [64]:
class Customer:
    def __init__(self, customer_id, customer_name):
        self.customer_id = customer_id
        self.customer_name = customer_name

In [71]:
import json

class CustomerSerializer:
    def serialize(self, data):
        if data is None:
            return None
        return json.dumps({
            'customer_id': data.customer_id,
            'customer_name': data.customer_name }).encode('utf-8')



In [72]:
customer_serializer = CustomerSerializer()

# Create a Kafka Producer
producer = KafkaProducer(
    bootstrap_servers = ['localhost:9092'],
    key_serializer=str.encode,
    value_serializer=customer_serializer.serialize
)

In [73]:
customer = Customer(123, 'John Doe')

In [81]:
future = producer.send('my_topic', key='customer_key', value=customer)

In [82]:
try:
    record_metadata = future.get(timeout=10)
    print(record_metadata)
except KafkaError as e:
    print(f"Failed to write message: {e}")

RecordMetadata(topic='my_topic', partition=0, topic_partition=TopicPartition(topic='my_topic', partition=0), offset=19, timestamp=1725615340082, log_start_offset=0, checksum=None, serialized_key_size=12, serialized_value_size=49, serialized_header_size=-1)
